In [18]:
import sys; sys.path.insert(0, '../..') # add parent folder path where lib folder is

%load_ext autoreload
%autoreload 2

from utils import helper, config, rayer, kaggle_dataset_helper

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from ml.models.base.tune.brisk_xgboost import BriskXGBoost

from sklearn.metrics import make_scorer
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import f1_score, roc_auc_score, precision_score, recall_score
from ml.models import common

import ray


import time
import pandas as pd
import pyarrow.fs

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
rayer.get_global_cluster()

In [20]:
df = helper.get_tokamat_dataset()
df = df.reset_index()

df = common.label_encode(df)
df = df.fillna(-1)

/mnt/c/Users/rwmas/GitHub/xai/python-asd/xai/notebooks/tune_sklearn/../../utils


In [21]:
potential_targets = ['WTOT', 'WTH', 'PLTH']
y = df[potential_targets[0]]

X = df[df.columns[~df.columns.isin(potential_targets)]]
X = X.drop(['TOK_ID', 'LCUPDATE', 'DATE', 'NEL', 'ENBI'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

In [22]:
r2_scoring = make_scorer(score_func=r2_score, greater_is_better=False)

In [24]:
objective = "count:poisson" # ["reg:squarederror", "count:poisson", "binary:logistic",  "binary:hinge" ]

brisk_xgboost_1 = BriskXGBoost(name='tune_brisk_xgboost_1', objective=objective,
                                                pred_class='regression',
                                                #n_estimators=100,
                                                score_func=None,
                                                #metric_func=mean_squared_error, 
                                                n_trials=2)

In [25]:
brisk_xgboost_1.fit(X_train, X_test, y_train, y_test)

(run pid=23398) /home/rayuser/.local/lib/python3.8/site-packages/ray/tune/execution/trial_runner.py:363: UserWarning: fail_fast='raise' detected. Be careful when using this mode as resources (such as Ray processes, file descriptors, and temporary files) may not be cleaned up properly. To use a safer mode, use fail_fast=True.
(run pid=23398)   warnings.warn(
(run pid=23398) 2022-12-06 09:59:39,395	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'cv': KFold(n_splits=3, random_state=None, shuffle=False), 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'scoring/score': <function _passthrough_scorer at 0x7fed3c5baca0>}
(run pid=23398) 2022-12-06 09:59:44,040	INFO tensorboardx.py:267 -- Removed the following hyperparameter values when logging to tensorboard: {'cv': KFold(n_splits=3, random_state=None, shuffle=False), 'early_stop_type': <EarlyStopping.NO_EARLY_STOP: 7>, 'scoring/score': <function _passthrough_scorer at 0x7fed3c5baca0>

In [26]:
brisk_xgboost_1.scores

[-1.2025579405296912e+26, -5.034220642428739e+26]

In [27]:
brisk_xgboost_1.score(X_test, y_test)

-5.034220642428739e+26